In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
final_clusters = pd.read_parquet('/data/uscuni-ulce/processed_data/clusters/clusters_v1.pq')[0]
from core.cluster_validation import get_color
final_colors = pd.DataFrame(get_color(final_clusters.values), final_clusters.values).drop_duplicates()
final_colors.loc[-1] = [255,255,255]

In [3]:

# region_id = 139196

# region_id = 69333  # prague
# region_id = 8707 # frankfurt / mainz
# region_id = 5883 # freiburtg
# region_id = 86873 # vienna
# region_id = 38679 # munich
# region_id = 55763 # berlin
# region_id = 107131 # krakow
# region_id = 99886 # bratislava

region_id = 151676 # vilnius

# region_id= 66593 # usti

# region_id = 34266 # nuremberg

In [4]:
etcs = gpd.read_parquet(f"/data/uscuni-ulce/processed_data/clusters/{region_id}_clusters.pq")

In [5]:
import lonboard
layer = lonboard.SolidPolygonLayer.from_geopandas(etcs, opacity=.7)

from sidecar import Sidecar
sc = Sidecar(title=f'Final Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatter)
with sc:
    display(m)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [6]:
from core.cluster_validation import get_color
layer.get_fill_color = final_colors.loc[etcs.final_without_noise].values.astype('uint8')

In [7]:
etcs.final.nunique()

17

In [8]:
# layer.get_fill_color = get_color(etcs.morph.factorize()[0])

In [9]:
# etcs[etcs.final == 27].explore()

In [32]:
cdata = pd.read_parquet('/data/uscuni-ulce/processed_data/clusters/cdata_v1.pq')

In [71]:
cdata.loc[[25, 30]].style.background_gradient(axis=0, cmap="BuGn")

,sdbAre,sdbPer,sdbCoA,ssbCCo,ssbCor,ssbSqu,ssbERI,ssbElo,mtbSWR,libNCo,ldbPWL,ltcBuA,mtbAli,mtbNDi,ltbIBD,stbSAl,sdsLen,sssLin,ldsMSL,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,sddAre,midRea,midAre,sdcLAL,sdcAre,sscCCo,sscERI,mtcWNe,mdcAre,ltcWRB,sicCAR,sdsAre,mibCou,mibAre,mibLen,mibElo,mibERI,mibCCo,mibLAL,mibFR,mibSCo,micBAD,midBAD
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25,251.435117,71.876777,0.000000,0.483769,8.000000,2.983651,0.936945,0.646995,0.357268,2.000000,515.892140,0.400000,3.014490,7.969107,11.566545,2.643032,141.296069,0.998303,115.260762,225.625000,222123.835278,0.086326,20.470200,0.238281,2.598608,3.000000,0.163819,0.698004,0.218783,0.061704,0.000003,0.006642,0.014229,53.856301,0.012346,120.323383,12052.554739,41.250000,39937.883044,42.548654,605.378252,0.476745,0.961668,0.059285,7010.393350,0.000104,0.428021,10094.061521,15.500000,5506.894053,859.969721,0.564153,0.357907,0.294370,148.166578,5.777471,5.777471,306.179772,241.372393
30,92.912700,42.268306,0.000000,0.502615,6.000000,2.808463,0.981340,0.634668,0.416959,1.500000,401.838837,0.366667,3.206593,3.851112,6.107542,4.738279,108.447027,0.996270,93.055048,314.000000,153428.100009,0.193057,14.024548,0.184659,3.941364,3.000000,0.142857,0.763073,0.128968,0.095238,0.000027,0.007739,0.015297,59.590857,0.000000,87.674881,7101.141934,58.500000,22328.057564,24.054620,197.315072,0.471377,0.964577,0.106068,2120.363359,0.000228,0.488466,6070.220100,18.500000,2217.712157,512.206726,0.585167,0.396957,0.318631,95.882791,4.153734,4.153734,90.383750,94.719572


In [9]:
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
region_hulls.explore()